# Initialization

In [2]:
import matplotlib.pyplot as plt
import xarray as xr

plt.rcParams['figure.figsize'] = [16, 3]

import sys
sys.path.append('/mnt/lustre/IAM851/jm1667/psc/python')
import psc

import logging
# logging.basicConfig(level=logging.DEBUG)

%config InlineBackend.figure_format = 'retina'
engine = "pscadios2"
species_names = ['e', 'i']

def openFile(fileName, step):
    return xr.open_dataset(path + f"{fileName}.{str(step).rjust(9,'0')}.bp", length=length, engine=engine, species_names=species_names)

In [65]:
case = "case1"
B = 1
res = 512
mods = ["", "_v-1", '_v0', '_dup', '_T0', '_v2'][-1]

path = f"/mnt/lustre/IAM851/jm1667/psc-runs/{case}/trials/B{B}_n{res}{mods}/"
length = {1:(1., .03, .03), 10:(1., .02, .02), .1:(1., .18, .18)}[B]
wholeSlice = slice(-length[1]/2, length[1]/2)
centerSlice = {10:slice(-.001, .001), 1:slice(-.003, .003)}[B]

with open(path + "params_record.txt") as records:
    fields_every = 200
    moments_every = 200
    for line in records:
        if line.startswith("nmax"):
            nmax = int(line.split()[1])
        elif line.startswith("fields_every"):
            fields_every = int(line.split()[1])
        elif line.startswith("moments_every"):
            moments_every = int(line.split()[1])
dt = openFile('pfd', fields_every)._attrs['time']
print(f"dt = {dt}")
print(f"nsteps in sim: {nmax}")
print(f"moments_every: {moments_every}")
print(f"fields_every: {fields_every}")
print(f"directory to save in: {case}/B{B}_n{res}{mods}")

dt = 0.00621480569402239
nsteps in sim: 1000000
moments_every: 200
fields_every: 200
directory to save in: case1/B1_n512


In [52]:
tstart = 0
tend = -1

if tend < 0:
    tend = (nmax // fields_every) * dt
def getStep(time: float, every: int):
    if time < 0:
        return nmax - (nmax % every)
    return int(time // dt) * every

print(f"t: [{tstart}, {tend}]")

t: [0, 20.716018980074637]


# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [30]:
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

def openFile(fileName, step):
    return xr.open_dataset(path + f"{fileName}.{str(step).rjust(9,'0')}.bp", length=length, engine=engine, species_names=species_names)

def prepData(data):
    return data[0,:,:].transpose()

class DataLoader:
    def __init__(self):
        self.rGrid = None
        self.horGrid = None
        self.verGrid = None
    def getDataAndTime(self, step: int):
        if step == 0 and param.skipFirst:
            file = openFile(param.fileName, {'pfd':fields_every, 'pfd_moments':moments_every}[param.fileName])
        else:
            file = openFile(param.fileName, step)

        if isinstance(param.varName, list):
            if param.combine == "magnitude":
                rawData = sum(prepData(file.__getattr__(var)) ** 2 for var in param.varName) ** .5
            else:
                rawHor = prepData(file.__getattr__(param.varName[0]))
                rawVer = prepData(file.__getattr__(param.varName[1]))
                if self.rGrid is None:
                    self.rGrid = (rawHor.y **2 + rawHor.z**2)**.5
                    self.horGrid = rawHor.y
                    self.verGrid = rawHor.z
                if param.combine == 'radial':
                    rawData = (rawHor * self.horGrid + rawVer * self.verGrid) / self.rGrid
                elif param.combine == 'azimuthal':
                    rawData = (-rawHor * self.verGrid + rawVer * self.horGrid) / self.rGrid
                rawData = rawData.fillna(0)
        else:
            rawData = prepData(file.__getattr__(param.varName))
        return param.coef * rawData, file._attrs['time']
    def loadData(self, stepiter):
        return [list(x) for x in zip(*[self.getDataAndTime(step) for step in stepiter])]

def setTitle(ax, param, time):
    ax.set_title(viewAdj + param.title + " (t={:.3f})".format(time))

class ParamMetadata:
    def __init__(self, title, vmin, vmax, colors, fileName, varName, coef=1, skipFirst=False, combine="magnitude"):
        self.title = title
        self.vmin = vmin
        self.vmax = vmax
        self.colors = colors
        self.fileName = fileName
        self.varName = varName
        self.coef = coef
        self.skipFirst = skipFirst
        self.combine = combine

In [66]:
paramId = 0
param = [
        ParamMetadata('Electron Density', 0, None, 'inferno', 'pfd_moments', 'rho_e', coef=-1),
        ParamMetadata('Ion Density', 0, None, 'inferno', 'pfd_moments', 'rho_i'),
        ParamMetadata('E^2', 0, None, 'inferno', 'pfd', ['ey_ec', 'ez_ec'], combine='magnitude'),
        ParamMetadata('Y-ial B', -3e-8, 3e-8, 'RdBu', 'pfd', 'hy_fc', skipFirst=True),
        ParamMetadata('X-ial B', -B*1.2, B*1.2, 'RdBu', 'pfd', 'hx_fc'),
        ParamMetadata('Y-ial J', -.0005, .0005, 'RdBu', 'pfd', 'jy_ec', skipFirst=True),
        ParamMetadata('Radial J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='radial'),
        ParamMetadata('Azimuthal J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='azimuthal'),
        ][paramId]
out_every = {'pfd':fields_every, 'pfd_moments':moments_every}[param.fileName]
print(f"out_every: {out_every}")
print(f"quantity: {param.title}")

out_every: 200
quantity: Electron Density


In [67]:
# load data
frames_skip = 4 # take 1 in `frames_skip` frames

stepStart = getStep(tstart, out_every)
stepEnd = getStep(tend, out_every)
nsteps = int((stepEnd - stepStart) / out_every / frames_skip)
print(f"Loading {nsteps} x {len(param.varName) if isinstance(param.varName, list) else 1} files...")
datas, times = DataLoader().loadData(range(getStep(tstart, out_every), getStep(tend, out_every), out_every * frames_skip))
print("Done.")

Loading 833 x 1 files...


KeyboardInterrupt: 

In [56]:
# slice data
activeSlice = [wholeSlice, centerSlice][0]

slicedDatas = [data.sel(y=activeSlice, z=activeSlice) for data in datas]
viewAdj = 'Central ' if activeSlice==centerSlice else ''
print(f"view: {viewAdj}= {activeSlice}")

vmax = param.vmax if not param.vmax is None else max(np.nanquantile(data.values, .9) for data in slicedDatas)
vmin = param.vmin if not param.vmin is None else min(np.nanquantile(data.values, .1) for data in slicedDatas)
if param.vmax is None and param.vmin is None:
    vmax = max(vmax, -vmin)
    vmin = -vmax

rGrid = (slicedDatas[0].y **2 + slicedDatas[0].z**2)**.5

view: = slice(-0.01, 0.01, None)


In [57]:
# view tstart
%matplotlib ipympl
fig, ax = plt.subplots()

im = ax.imshow(slicedDatas[0], cmap=param.colors, vmin=vmin, vmax=vmax, origin='lower', extent=(activeSlice.start, activeSlice.stop, activeSlice.start, activeSlice.stop))
ax.set_xlabel('y')
ax.set_ylabel('z')
setTitle(ax, param, times[0])
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
fig.colorbar(im, ax=ax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Stability Plot

In [58]:
%matplotlib ipympl
def norm(x):
    return xr.apply_ufunc(np.linalg.norm, x, input_core_dims=[['y', 'z']])

normsOfDiffs = [norm(data - slicedDatas[0]) for data in slicedDatas]

plt.xlabel("Time")
plt.ylabel("2-Norm of Difference")
plt.title("Stability of " + viewAdj + param.title)

plt.plot(times, normsOfDiffs)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FFT

In [21]:
import xrft
import matplotlib

In [59]:
da = xr.concat(datas, 't').assign_coords(t=("t", times))
Fda = da.coarsen(y=2,z=2).mean()
for dim in da.dims:
    print(f"DFTing {dim}...")
    Fda = xrft.dft(Fda, dim=dim, true_phase=True, true_amplitude=True)
print("Done.")

halfAxis = len(Fda.coords['freq_y']) // 2

DFTing t...
DFTing z...
DFTing y...
Done.


In [60]:
%matplotlib ipympl
fig, (ax1,ax2) = plt.subplots(figsize=(12,4), nrows=1, ncols=2)

pltme = Fda[:,:,halfAxis]

pltme.real.plot(ax=ax1, vmax=np.nanquantile(pltme.real.data, 0.999))
ax1.set_title("Real Part")
pltme.imag.plot(ax=ax2, vmax=np.nanquantile(pltme.imag.data, 0.999))
ax2.set_title("Imaginary Part")
fig.suptitle("DFT of " + viewAdj + param.title, fontsize=16)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
%matplotlib ipympl
fig, ax = plt.subplots()

pltme = (Fda[:,:,halfAxis].real **2 + Fda[:,:,halfAxis].imag**2).sel(freq_t=slice(0,None), freq_z=slice(0,None)) ** .5

pltme.real.plot(ax=ax, vmax=np.nanquantile(pltme.real.data,1), norm=matplotlib.colors.LogNorm())
ax.set_title("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Radial Dependence

In [62]:
# Bin values of k 
kgrid = (Fda.freq_y ** 2 + Fda.freq_z**2) ** .5
maxK = kgrid.max()
kstep = maxK / 64

def getMean(data, k):
    kslice = data.where((k <= kgrid) & (kgrid < k + kstep))
    return kslice.mean().item()

ks = np.arange(0, maxK, kstep)
def getOmK(omKyKz):
    return xr.DataArray([[getMean(omKyKz[iom,:,:], k) for k in ks] for iom in range(len(omKyKz.freq_t))], dims=['om', 'k'], coords=[omKyKz.freq_t.data * 2*np.pi, ks * 2*np.pi])

In [63]:
omK = getOmK(Fda)

In [64]:
%matplotlib ipympl
fig, ax = plt.subplots()

omMax = None
kMax = None
pltme = abs(omK.sel(om=slice(0, omMax), k=slice(0, kMax)))

# pltme.real.plot(ax=ax, cmap='afmhot', norm=matplotlib.colors.LogNorm())
pltme.real.plot(ax=ax, vmax=np.nanquantile(pltme.real.data, .999), cmap='viridis')
ax.set_title(f"Spectrum for {viewAdj}{param.title} (t: [{tstart}, {tend.round(1)}])")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# DFT at y=0

In [61]:
zFda = da.coarsen(y=2,z=2).mean()
for dim in da.dims:
    if dim != 'y':
        print(f"DFTing {dim}...")
        zFda = xrft.dft(zFda, dim=dim, true_phase=True, true_amplitude=True)
print("Done.")

# halfAxis = len(Fda.coords['freq_y']) // 2

DFTing t...
DFTing z...
Done.


In [79]:
%matplotlib ipympl
fig, ax = plt.subplots()

pltme = abs(zFda.sel(y=0, method='nearest').sel(freq_z=slice(0, None), freq_t=slice(0, None)))
pltme.plot(ax=ax, vmax=np.nanquantile(pltme.real.data, .999))#, norm=matplotlib.colors.LogNorm())
ax.set_title("Magnitude")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …